# Evaluación de la Esperanza de Vida en Europa (1900–2023)

En este proyecto he desarrollado un análisis espacial de la esperanza de vida en Europa entre 1900 y 2023, conectando un CSV con información por país (código ISO, año y esperanza de vida media) con un shapefile geográfico.

In [1]:
# INSTALACIONES NECESARIAS

! pip install pandas geopandas


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# IMPORTACIONES NECESARIAS

import pandas as pd
import geopandas as gpd

In [3]:
life_e = pd.read_csv("life-expectancy.csv")
life_e.head()

,Entity,Code,Year,Period life expectancy at birth
0,Afghanistan,AFG,1950,28.1563
1,Afghanistan,AFG,1951,28.5836
2,Afghanistan,AFG,1952,29.0138
3,Afghanistan,AFG,1953,29.4521
4,Afghanistan,AFG,1954,29.6975


In [4]:
# Tamaño del csv:

life_e.shape

(21565, 4)

Para estre proyecto, se necesitan recopilar sólo los datos pertenecientes a Europa.
Para estructurar este diccionario, es necesario conocer todos los valores únicos de mi columna Entity
y con el apoyo de inteligencia artificial, se organizan y clasifican por continente.

In [5]:
#  Diccionario país -> continente (mapa inicial)
pais_a_continente = {
    # Asia
    'Afghanistan': 'Asia', 'Armenia':'Asia', 'Azerbaijan':'Asia',
    'Bahrain':'Asia', 'Bangladesh':'Asia', 'Bhutan':'Asia', 'Brunei':'Asia',
    'Cambodia':'Asia', 'China':'Asia', 'Georgia':'Asia', 'India':'Asia',
    'Indonesia':'Asia', 'Iran':'Asia', 'Iraq':'Asia', 'Israel':'Asia',
    'Japan':'Asia', 'Jordan':'Asia', 'Kazakhstan':'Asia', 'Kuwait':'Asia',
    'Kyrgyzstan':'Asia', 'Laos':'Asia', 'Lebanon':'Asia', 'Malaysia':'Asia',
    'Maldives':'Asia', 'Mongolia':'Asia', 'Myanmar':'Asia', 'Nepal':'Asia',
    'North Korea':'Asia', 'Oman':'Asia', 'Pakistan':'Asia', 'Palestine':'Asia',
    'Philippines':'Asia', 'Qatar':'Asia', 'Saudi Arabia':'Asia', 'Singapore':'Asia',
    'South Korea':'Asia', 'Sri Lanka':'Asia', 'Syria':'Asia', 'Taiwan':'Asia',
    'Tajikistan':'Asia', 'Thailand':'Asia', 'East Timor':'Asia', 'Turkey':'Asia',
    'Turkmenistan':'Asia', 'United Arab Emirates':'Asia', 'Uzbekistan':'Asia',
    'Vietnam':'Asia', 'Yemen':'Asia',

    # Europe
    'Albania':'Europe', 'Andorra':'Europe', 'Austria':'Europe', 'Belarus':'Europe',
    'Belgium':'Europe', 'Bosnia and Herzegovina':'Europe', 'Bulgaria':'Europe',
    'Croatia':'Europe', 'Cyprus':'Europe', 'Czechia':'Europe', 'Denmark':'Europe',
    'Estonia':'Europe', 'Finland':'Europe', 'France':'Europe', 'Germany':'Europe',
    'Greece':'Europe', 'Hungary':'Europe', 'Iceland':'Europe', 'Ireland':'Europe',
    'Italy':'Europe', 'Kosovo':'Europe', 'Latvia':'Europe', 'Liechtenstein':'Europe',
    'Lithuania':'Europe', 'Luxembourg':'Europe', 'Malta':'Europe', 'Moldova':'Europe',
    'Monaco':'Europe', 'Montenegro':'Europe', 'Netherlands':'Europe', 'North Macedonia':'Europe',
    'Norway':'Europe', 'Poland':'Europe', 'Portugal':'Europe', 'Romania':'Europe',
    'Russia':'Europe', 'San Marino':'Europe', 'Serbia':'Europe', 'Slovakia':'Europe',
    'Slovenia':'Europe', 'Spain':'Europe', 'Sweden':'Europe', 'Switzerland':'Europe',
    'Ukraine':'Europe', 'United Kingdom':'Europe', 'England and Wales':'Europe',
    'Scotland':'Europe', 'Northern Ireland':'Europe', 'Gibraltar':'Europe',
    'Faroe Islands':'Europe',

    # Africa
    'Algeria':'Africa', 'Angola':'Africa', 'Benin':'Africa', 'Botswana':'Africa',
    'Burkina Faso':'Africa', 'Burundi':'Africa', 'Cameroon':'Africa', 'Cape Verde':'Africa',
    'Central African Republic':'Africa', 'Chad':'Africa', 'Comoros':'Africa',
    'Congo':'Africa', 'Democratic Republic of Congo':'Africa', 'Djibouti':'Africa',
    'Egypt':'Africa', 'Equatorial Guinea':'Africa', 'Eritrea':'Africa', 'Eswatini':'Africa',
    'Ethiopia':'Africa', 'Gabon':'Africa', 'Gambia':'Africa', 'Ghana':'Africa',
    'Guinea':'Africa', 'Guinea-Bissau':'Africa', "Cote d'Ivoire":'Africa', 'Kenya':'Africa',
    'Lesotho':'Africa', 'Liberia':'Africa', 'Libya':'Africa', 'Madagascar':'Africa',
    'Malawi':'Africa', 'Mali':'Africa', 'Mauritania':'Africa', 'Mauritius':'Africa',
    'Mayotte':'Africa', 'Morocco':'Africa', 'Mozambique':'Africa', 'Namibia':'Africa',
    'Niger':'Africa', 'Nigeria':'Africa', 'Rwanda':'Africa', 'Sao Tome and Principe':'Africa',
    'Senegal':'Africa', 'Seychelles':'Africa', 'Sierra Leone':'Africa', 'Somalia':'Africa',
    'South Africa':'Africa', 'South Sudan':'Africa', 'Sudan':'Africa', 'Tanzania':'Africa',
    'Togo':'Africa', 'Tunisia':'Africa', 'Uganda':'Africa', 'Western Sahara':'Africa',
    'Zambia':'Africa', 'Zimbabwe':'Africa',

    # Americas
    'Anguilla':'Americas', 'Antigua and Barbuda':'Americas', 'Argentina':'Americas',
    'Aruba':'Americas', 'Bahamas':'Americas', 'Barbados':'Americas', 'Belize':'Americas',
    'Bermuda':'Americas', 'Bolivia':'Americas', 'Bonaire Sint Eustatius and Saba':'Americas',
    'Brazil':'Americas', 'British Virgin Islands':'Americas', 'Canada':'Americas',
    'Cayman Islands':'Americas', 'Chile':'Americas', 'Colombia':'Americas', 'Costa Rica':'Americas',
    'Cuba':'Americas', 'Curacao':'Americas', 'Dominica':'Americas', 'Dominican Republic':'Americas',
    'Ecuador':'Americas', 'El Salvador':'Americas', 'French Guiana':'Americas', 'Greenland':'Americas',
    'Grenada':'Americas', 'Guadeloupe':'Americas', 'Guatemala':'Americas', 'Guyana':'Americas',
    'Haiti':'Americas', 'Honduras':'Americas', 'Jamaica':'Americas', 'Martinique':'Americas',
    'Mexico':'Americas', 'Montserrat':'Americas', 'Nicaragua':'Americas', 'Panama':'Americas',
    'Paraguay':'Americas', 'Peru':'Americas', 'Puerto Rico':'Americas', 'Saint Barthelemy':'Americas',
    'Saint Helena':'Americas', 'Saint Kitts and Nevis':'Americas', 'Saint Lucia':'Americas',
    'Saint Martin (French part)':'Americas', 'Saint Pierre and Miquelon':'Americas',
    'Saint Vincent and the Grenadines':'Americas', 'Suriname':'Americas',
    'Trinidad and Tobago':'Americas', 'United States':'Americas',
    'United States Virgin Islands':'Americas', 'Uruguay':'Americas', 'Venezuela':'Americas',

    # Oceania
    'American Samoa':'Oceania', 'Australia':'Oceania', 'Fiji':'Oceania', 'Guam':'Oceania',
    'Kiribati':'Oceania', 'Marshall Islands':'Oceania', 'Micronesia (country)':'Oceania',
    'Nauru':'Oceania', 'New Caledonia':'Oceania', 'New Zealand':'Oceania',
    'Niue':'Oceania', 'Northern Mariana Islands':'Oceania', 'Palau':'Oceania',
    'Papua New Guinea':'Oceania', 'Samoa':'Oceania', 'Solomon Islands':'Oceania',
    'Tokelau':'Oceania', 'Tonga':'Oceania', 'Tuvalu':'Oceania', 'Vanuatu':'Oceania',
    'Wallis and Futuna':'Oceania',

    # Antártida
    'Antarctica':'Antarctica'
}

# 3️⃣ Crear nueva columna 'continent'
life_e['continent'] = life_e['Entity'].map(pais_a_continente)

# 4️⃣ Guardar en un nuevo CSV
life_e.to_csv("life_expectancy_con_continent.csv", index=False)

# 5️⃣ Mostrar ejemplo
print(life_e[['Entity','continent']].head(10))


        Entity continent
0  Afghanistan      Asia
1  Afghanistan      Asia
2  Afghanistan      Asia
3  Afghanistan      Asia
4  Afghanistan      Asia
5  Afghanistan      Asia
6  Afghanistan      Asia
7  Afghanistan      Asia
8  Afghanistan      Asia
9  Afghanistan      Asia


In [6]:
# Ahora mi csv está formado también por la columna continente
 
life_e.head()

,Entity,Code,Year,Period life expectancy at birth,continent
0,Afghanistan,AFG,1950,28.1563,Asia
1,Afghanistan,AFG,1951,28.5836,Asia
2,Afghanistan,AFG,1952,29.0138,Asia
3,Afghanistan,AFG,1953,29.4521,Asia
4,Afghanistan,AFG,1954,29.6975,Asia


In [7]:
# Necesitamos filtrar nuestro CSV por continent --> Europe

life_europa = life_e[life_e['continent'] == "Europe"]

In [8]:
# Y lo guardamos en un nuevo csv

life_europa.to_csv("life_europa.csv")

In [9]:
"""Función para obtener un informe rápido de nulos y duplicados
   de un DataFrame:"""

def initial_report(df): #Esta función actua directamente con el DF como argumento.

    # Calculamos el porcentaje de nulos
    porcentaje_nulos = df.isna().sum()/df.shape[0]*100

    # Tipo de dato por columna:
    tipos_datos = df.dtypes

    # Verificación de duplicados
    duplicados = df.duplicated().sum()
    if duplicados == 0: # Ponemos un condicional
        mensaje_duplicados = "No hay duplicados"
    else:
        mensaje_duplicados = f"Hay {duplicados} duplicados"

    # Unimos la información de los nulos y el tipo de dato de cada columna
    info_columnas = pd.DataFrame({'Tipo de Dato': tipos_datos,'Porcentaje de Nulos': porcentaje_nulos})
    # Crear string para el informe
    info_str = info_columnas.to_string()

    # Este es el informe visual que recibimos:
    informe = f"""
    ========= Informe de Datos 📋 ==========
    
    📌 % Nulos y tipo de dato:
    -------------------------------------
    {info_str}
    -------------------------------------
    🟰Duplicados:
    -------------------------------------
    {mensaje_duplicados}
    =====================================
    """
    # Imprimir el informe
    print(informe)

In [10]:
initial_report(life_europa)


    ========= Informe de Datos 📋 ==========
    
    📌 % Nulos y tipo de dato:
    -------------------------------------
                                    Tipo de Dato  Porcentaje de Nulos
Entity                                object              0.00000
Code                                  object              4.45383
Year                                   int64              0.00000
Period life expectancy at birth      float64              0.00000
continent                             object              0.00000
    -------------------------------------
    🟰Duplicados:
    -------------------------------------
    No hay duplicados
    


In [11]:
# Tengo nulos en CODE, que se puede solucionar y no hay duplicados

#¿ Dónde tengo nulos en code? ¿Son de mi interés para el estudio?

# Mostrar las filas donde 'Code' es nulo
nulos_code = life_e[life_e['Code'].isna()]

# Ver solo la columna 'Entity' de esos nulos
print(nulos_code['Entity'].unique())

['Africa' 'Americas' 'Asia' 'England and Wales' 'Europe'
 'High-and-upper-middle-income countries' 'High-income countries'
 'Land-locked Developing Countries (LLDC)'
 'Latin America and the Caribbean' 'Least developed countries'
 'Less developed regions' 'Less developed regions, excluding China'
 'Less developed regions, excluding least developed countries'
 'Low-and-Lower-middle-income countries' 'Low-and-middle-income countries'
 'Low-income countries' 'Lower-middle-income countries'
 'Middle-income countries' 'More developed regions'
 'No income group available' 'Northern America' 'Northern Ireland'
 'Oceania' 'Scotland' 'Small Island Developing States (SIDS)'
 'Upper-middle-income countries']


La lista resultante no son países individuales, sino regiones, agrupaciones o subdivisiones
Estas entidades no tienen código ISO, por eso Code es nulo. No se considera relevante para el proyecto

In [12]:
# Sólo nos interesan los años, de 1900 en adelante

life_europa["Year"].unique()

array([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
       1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 1870, 1881, 1891,
       1901, 1903, 1911, 1931, 1947, 1948, 1949, 1900, 1841, 1842, 1843,
       1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854,
       1855, 1856, 1857, 1858, 1859, 1860, 1861, 1862, 1863, 1864, 1865,
       1866, 1867, 1868, 1869, 1871, 1872, 1873, 1874, 1875, 1876, 1877,
       1878, 1879, 1880, 1882, 1883, 1884, 1885, 1886, 1887, 1888, 1889,
       1890, 1892, 1893, 1894, 1895, 1896, 1897, 1898, 1899, 1902, 1904,
       1905, 1906, 1907, 1908, 1909, 1910, 1912, 19

In [13]:
# Asegurarnos de que la columna 'Year' sea numérica
life_europa['Year'] = pd.to_numeric(life_europa['Year'], errors='coerce')

# Filtrar solo años >= 1800
life_europa = life_europa[life_europa['Year'] >= 1900].copy()

# Verificar
print(life_europa['Year'].min(), life_europa['Year'].max())

1900 2023


C:\Users\isaw9\AppData\Local\Temp\ipykernel_14472\2715625548.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  life_europa['Year'] = pd.to_numeric(life_europa['Year'], errors='coerce')


In [14]:
# Definir los años que queremos conservar
años_clave = [1900, 1918, 1950, 1975, 2000, 2023]

# Filtrar solo esas filas
life_europa_years = life_europa[life_europa['Year'].isin(años_clave)].copy()

In [15]:
# Resumen rápido de "Period life expectancy at birth" en España en esos años
life_europa_years[(life_europa_years['Code'] == 'ESP')]

,Entity,Code,Year,Period life expectancy at birth,continent
18088,Spain,ESP,1900,34.8000,Europe
18099,Spain,ESP,1918,30.3600,Europe
18131,Spain,ESP,1950,61.7542,Europe
18156,Spain,ESP,1975,73.4918,Europe
18181,Spain,ESP,2000,79.4123,Europe
18204,Spain,ESP,2023,83.6703,Europe


In [16]:
life_europa_years.to_csv("life_europa_years.csv")

In [17]:
# 1️⃣ Leer shapefile
shp = gpd.read_file("mapa_mundo/mapa_mundo.shp")

# Ver las columnas disponibles
print(shp.columns)
print(shp.head())

# 2️⃣ Filtrar solo Europa

# Si no hay columna continente, puedes usar lista de países europeos
paises_europa = [
    'Albania','Andorra','Austria','Belarus','Belgium','Bosnia and Herzegovina',
    'Bulgaria','Croatia','Cyprus','Czechia','Denmark','Estonia','Finland',
    'France','Germany','Greece','Hungary','Iceland','Ireland','Italy','Kosovo',
    'Latvia','Liechtenstein','Lithuania','Luxembourg','Malta','Moldova','Monaco',
    'Montenegro','Netherlands','North Macedonia','Norway','Poland','Portugal',
    'Romania','Russia','San Marino','Serbia','Slovakia','Slovenia','Spain','Sweden',
    'Switzerland','Ukraine','United Kingdom','England and Wales','Scotland','Northern Ireland'
]

europa_shp = shp[shp['NAME'].isin(paises_europa)].copy()  # 'NAME' = columna con nombres de países

# 3️⃣ Unir con tu CSV de esperanza de vida
life_csv = pd.read_csv("life_europa_years.csv")  # tu CSV limpio

# Merge usando columna de código ISO o nombre
europa_shp = europa_shp.merge(life_csv, left_on='ISO_A3', right_on='Code', how='left')

# 4️⃣ Guardar shapefile nuevo
europa_shp.to_file("europa_life_expectancy.shp")

print("¡Shapefile listo! Ahora lo puedes abrir en QGIS con solo Europa y tus datos de esperanza de vida.")


Index(['fid', 'iso_a2', 'NAME', 'FIPS_10_', 'ISO_A3', 'WB_A2', 'WB_A3',
       'geometry'],
      dtype='object')
   fid iso_a2         NAME FIPS_10_ ISO_A3 WB_A2 WB_A3  \
0  1.0     NO       Norway       NO    NOR    NO   NOR   
1  2.0     SE       Sweden       SW    SWE    SE   SWE   
2  3.0     DE      Germany       GM    DEU    DE   DEU   
3  4.0     NL  Netherlands       NL    NLD    NL   NLD   
4  8.0     RU       Russia       RS    RUS    RU   RUS   

                                            geometry  
0  MULTIPOLYGON (((3.45729 -54.39007, 3.48666 -54...  
1  MULTIPOLYGON (((15.70533 56.1164, 15.7269 56.1...  
2  MULTIPOLYGON (((6.79811 53.60444, 6.72242 53.5...  
3  MULTIPOLYGON (((-68.21154 12.22809, -68.19001 ...  
4  MULTIPOLYGON (((47.94386 45.49958, 47.91774 45...  
¡Shapefile listo! Ahora lo puedes abrir en QGIS con solo Europa y tus datos de esperanza de vida.


C:\Users\isaw9\AppData\Local\Temp\ipykernel_14472\1255458508.py:30: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  europa_shp.to_file("europa_life_expectancy.shp")
c:\Users\isaw9\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Unnamed: 0' to 'Unnamed_ 0'
  ogr_write(
c:\Users\isaw9\anaconda3\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Period life expectancy at birth' to 'Period lif'
  ogr_write(
